# Docx functions

In [1]:
# !pip install python-docx
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_COLOR_INDEX
from docx.shared import Pt
import shutil

In [1]:
def create_file_based_template (template_path, document_path):
    shutil.copy(template_path,document_path)

def append_paragraph_in_document (document_path, text):     
    document = Document(document_path)
    document.add_paragraph(text)
    document.save(document_path) 
    
def append_header_in_document (document_path, level, header_text):     
    document = Document(document_path)
    document.add_heading(header_text, level=level)
    document.save(document_path) 
    
def append_fig_in_document (document_path, figure_path, figure_label):
    document = Document(document_path)
    document.add_picture(figure_path) #, width=Inches(5.9))
    document.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    fig_label = document.add_paragraph()
    fig_label_header=  fig_label.add_run('Figure. ')
    fig_label_header.bold = True
    fig_label.add_run(figure_label)
    document.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    document.save(document_path) 
    
def append_dataframe_in_document (document_path, dataframe):
    document = Document(document_path)
    #fixing the index
    dataframe = dataframe.reset_index()
    dataframe.index +=1
    dataframe= dataframe.rename(columns={"index":"#"})
    #creating the table
    t = document.add_table(dataframe.shape[0]+1, dataframe.shape[1],style='mvt3')
    t.alignment = WD_TABLE_ALIGNMENT.CENTER
    # add the header rows.
    for j in range(dataframe.shape[-1]):
        t.cell(0,j).text = dataframe.columns[j]
    # add the rest of the data frame
    for i in range(dataframe.shape[0]):
        for j in range(dataframe.shape[-1]):
            t.cell(i+1,j).text = str(dataframe.values[i,j])
   
    for row in t.rows:
        for cell in row.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                for run in paragraph.runs:
                    font = run.font
                    font.size= Pt(9)
                    font.name = 'Calibri'

    document.save(document_path)

def search_n_replace_4word (document_path, search_text, replace_text):
    document = Document(document_path)
    for paragraph in document.paragraphs:
        if search_text in paragraph.text:
            inline = paragraph.runs
            # Loop added to work with runs (strings with same style)
            for i in range(len(inline)):
                if search_text in inline[i].text:
                    text = inline[i].text.replace(search_text, str(replace_text))
                    inline[i].text = text
    document.save(document_path)
    
def search_n_replace_4paragraph (document_path, search_text, replace_text, keep_anchor=False):
    document = Document(document_path)
    for paragraph in document.paragraphs:
        if search_text in paragraph.text:
            paragraph.text = replace_text
            if keep_anchor == True:
                paragraph._p.addnext(document.add_paragraph(search_text)._p)            
    document.save(document_path)
    
def search_n_replace_4figure (document_path, search_text, figure_path, keep_anchor=False):
    document = Document(document_path)
    for paragraph in document.paragraphs:
        if search_text in paragraph.text:
            print(' - FOUND & REPLACED:',search_text,'\n')
            paragraph.text = ''
            paragraph.add_run().add_picture(figure_path, width=Inches(5.11))
            document.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

            if keep_anchor == True:
                paragraph._p.addnext(document.add_paragraph(search_text)._p)            
    document.save(document_path)
    
def search_n_replace_4header (document_path, search_text, replace_text, level, keep_anchor=False):
    document = Document(document_path)
    for paragraph in document.paragraphs:
        if search_text in paragraph.text:
            paragraph.text = ''
            
            h = document.add_heading(replace_text, level=level)._p
            paragraph._p.addnext(h)
            
            if keep_anchor == True:
                h.addnext(document.add_paragraph(search_text)._p)            
    document.save(document_path)
    
    
def search_n_replace_4dataframe (document_path, search_text, dataframe, keep_anchor=False):
    document = Document(document_path)

    dataframe = dataframe.reset_index()
    dataframe= dataframe.rename(columns={"index":"#"})
    dataframe['#'] +=1
      
    for paragraph in document.paragraphs:
        if search_text in paragraph.text:
            print(' - FOUND & REPLACED:',search_text,'\n')
            paragraph.text = ""
            
            #creating the table
            t = document.add_table(dataframe.shape[0]+1, dataframe.shape[1],style='mvt3')
            t.alignment = WD_TABLE_ALIGNMENT.CENTER
            # add the header rows.
            for j in range(dataframe.shape[-1]):
                t.cell(0,j).text = dataframe.columns[j]
            # add the rest of the data frame
            for i in range(dataframe.shape[0]):
                for j in range(dataframe.shape[-1]):
                    t.cell(i+1,j).text = str(dataframe.values[i,j])

#             for row in t.rows:
#                 for cell in row.cells:
#                     paragraphs = cell.paragraphs
#                     for paragraph in paragraphs:
#                         for run in paragraph.runs:
#                             font = run.font
#                             font.size= Pt(9)
#                             font.name = 'Calibri'
            
            ## Substitution
            tbl, p = t._tbl, paragraph._p
            
            p.addnext(tbl)
            
            if keep_anchor:
                tbl.addnext(document.add_paragraph(search_text)._p)
            break

    document.save(document_path)

def pagebreak (document_path):
    document = Document(document_path)
    document.add_page_break()
    document.save(document_path)

## Example of usage 

In [ ]:
# customer_name = 'Hame Hame'

# ## Copying from the template
# template_path = './TEMPLATE.docx'
# document_path = './output/report_'+customer_name+'.docx'
# create_file_based_template(template_path, document_path)

# ## Appending a paragraph at the end of the document
# append_paragraph_in_document (document_path, 'hame text paragraph')

# ## Appending headers at the end of the document
# append_header_in_document (document_path, 1, 'Header 1')
# append_header_in_document (document_path, 2, 'Header 2')
# append_header_in_document (document_path, 3, 'Header 3')

# ## Appending a figure at the end of the document
# append_fig_in_document (document_path, 'name.png', 'test pie')

# ## Appending a dataframe at the end of the document
# df_example = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
# append_dataframe_in_document (document_path, df_example)

# ## Searching and Replacing text
# search_n_replace (document_path, '<<CUSTOMERNAME>>', 'Hame')

# ## Openning the document
# !open $document_path

In [ ]:
def fillin_report (document_path,document_section,df_assessment,screenshot_name="",num_figures=""):
    #Adding the header accordingly to the style level
    if df.loc[section_num]['Type'] == 'DOCUMENT_HEADER1':
        document.add_heading(df.loc[section_num]['Aspect'], level=1)
        ### Adding the Description subsection
        document.add_paragraph(df.loc[section_num]['Description'])
#         last_paragraph = document.paragraphs[-1] 
#         last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

    elif df.loc[section_num]['Type'] == 'DOCUMENT_HEADER2':
        document.add_heading(df.loc[section_num]['Aspect'], level=2)
        ### Adding the Description subsection
        document.add_paragraph(df.loc[section_num]['Description'])
#         last_paragraph = document.paragraphs[-1] 
#         last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

    else:
        document.add_heading(df.loc[section_num]['Aspect'], level=2)
        ### Adding the Description subsection
        document.add_paragraph(df.loc[section_num]['Description'])
#         last_paragraph = document.paragraphs[-1] 
#         last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

        ### Adding the Assessment subsection
        document.add_heading('Assessment', level=3)
        
        if not pd.isnull(df.loc[section_num]['Assessment_Begin']):
            document.add_paragraph(df.loc[section_num]['Assessment_Begin'])

        ### Adding the df_assessment in the report
        if not df_assessment.empty:
            df_assessment = df_assessment.reset_index()
            df_assessment= df_assessment.rename(columns={"index":"#"})
            t = document.add_table(df_assessment.shape[0]+1, df_assessment.shape[1])
            # add the header rows.
            for j in range(df_assessment.shape[-1]):
                t.cell(0,j).text = df_assessment.columns[j]
            # add the rest of the data frame
            for i in range(df_assessment.shape[0]):
                for j in range(df_assessment.shape[-1]):
                    t.cell(i+1,j).text = str(df_assessment.values[i,j])
        
        ### Adding the Assessment subsection
        if not pd.isnull(df.loc[section_num]['Assessment_End']):
            document.add_paragraph().add_run(df.loc[section_num]['Assessment_End']).font.highlight_color = WD_COLOR_INDEX.YELLOW
                
        ### Adding the figures/printscreen in the Assessment subsection    
        if screenshot_name:
            i = 1
            while (i <= num_figures) :
                document.add_picture(screenshot_name+'_'+str(i)+'.png', width=Inches(5.9))
                last_paragraph = document.paragraphs[-1] 
                last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                i +=1     

        ### Adding the recommendation subsection
        document.add_heading('Recommendation', level=3)
        p = document.add_paragraph(df.loc[section_num]['Recommendation'])
        last_paragraph = document.paragraphs[-1] 
#         last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

    ### Saving the document    
    document.save(report_path)  